<a href="https://colab.research.google.com/github/vietai/back_translate/blob/master/colabs/Sentiment_Analysis_%2B_Back_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this colab, we walk you through how to use our code in `vietai/back_translate` to augment an example of a Vietnamese dataset. The dataset is relatively small: it consists of only 27,000 supervised examples in its training set - perfect for some experiments on data augmentation!

For this task, we have already trained vi --> en and en --> vi translation models and placed them on Google Cloud Storage. There is no need to train new ones unless you have more data or data that fits the downstream task  better -- more on this later.



**MIT License**

Copyright (c) [2019] [Trieu H. Trinh](https://thtrieu.github.io/)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Install dependencies.

* `gcsfuse` to mount this colab to the Google Cloud Storage. Alternatively you can also mount the colab to Google Drive if you do not have access to GCS.
* `tensor2tensor`: a library with all necessary tools to perform training/inference.

In [0]:
# Mount the bucket to colab, so that python package os can access to it.
# First we install gcsfuse to be able to mount Google Cloud Storage with Colab.
print('\nInstalling gcsfuse')
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Now we install tensor2tensor
!pip install -q -U tensor2tensor
print('All done.')


Installing gcsfuse
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   659  100   659    0     0  12673      0 --:--:-- --:--:-- --:--:-- 12673
OK
26 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 26 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...
     |████████████████████████████████| 1.4MB 6.9MB/s 
     |█████████████

# (Maybe) Connect to 8 TPUs and mount to Cloud Storage.

You will be asked to authenticate your google account (gmail) through the browser. This is needed to have access to storages (google cloud/ google drive). Otherwise, the default Colab local storage will be used. This storage is wiped the moment you refresh the browser/ restart the runtime - which is not good for tasks that require long running and checkpointing.

In [0]:
import tensorflow as tf
import os
import re
import json
import pprint
from google_drive_downloader import GoogleDriveDownloader as gdd
import pandas as pd
from google.colab import auth

auth.authenticate_user()

# In case of TPU runtime, we need to authenticate with google cloud
if 'COLAB_TPU_ADDR' in os.environ:
  use_tpu = True
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    # Upload credentials to TPU.

    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

  print('TPU devices:')
  pprint.pprint(devices)
else:
  use_tpu, tpu_address = False, ''

Now we mount.

In [0]:
# Now we mount the local storage to the google cloud bucket.
bucket = 'vien-translation'
print('Mounting bucket {} to local.'.format(bucket))
mount_point = '/content/{}'.format(bucket)

if not os.path.exists(mount_point):
  tf.gfile.MakeDirs(mount_point)

!fusermount -u $mount_point
!gcsfuse --implicit-dirs $bucket $mount_point
!ls $mount_point

Mounting bucket vien-translation to local.
fusermount: failed to unmount /content/vien-translation: Invalid argument
Using mount point: /content/vien-translation
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.
checkpoints  data  foody_sa  raw  tmp


# Download Foody Sentiment Analysis dataset

Here we use data crawled from Foody.vn with 30,000 annotated examples by `streetcodevn.com`, with a half-half ratio of negative to positive examples. This dataset is used as an assignment in our VietAI Foundation class.

We split this dataset into 27,000 examples for training and the remaining 3,000 examples for testing. Use the following script to download the data. After downloading you will see `train.csv` and `test.csv`, which correspond to the train and test split.

In [0]:
foody_dir = os.path.join(mount_point, 'foody_sa')
gdd.download_file_from_google_drive(file_id='1bBcVHVPtXbzgwpkDYXC4zfTI8mek4Xsu', 
                                    dest_path='{}/assignment4-data.zip'.format(foody_dir), 
                                    unzip=True)
foody_data_dir = os.path.join(foody_dir, 'data')
!ls $foody_data_dir/t*.csv

/content/vien-translation/foody_sa/data/test.csv
/content/vien-translation/foody_sa/data/train.csv


# Examine the dataset

In [0]:
train_csv = os.path.join(foody_data_dir, 'train.csv')
train_df = pd.read_csv(train_csv)
train_df

,id,text,class
0,14feb4e5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy...",1
1,9c6f6036,Đến quán vào tối chủ_nhật . Có band hát . Khá ...,0
2,0f462cfe,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi g...,0
3,741fd21b,"Ko gian bé_tí , quán chật_chội , đông người nê...",0
4,5cd6dbbc,"Khi mình order , đặt bánh thì nhận được sự tiế...",1
5,a9b0679b,An ở đây lần thứ n mà giờ mới review . Quán lú...,0
6,d0919c4a,Quán rất đẹp . Nhạc rất hay . Rất art ! ! Bước...,1
7,a7dd3e51,Gemini thì nhạc bao hay òy : 3 Dj đánh là lên ...,1
8,4dc3df78,Mình và bạn mình vào đây ngày 8/3 . Mình thấy ...,0
9,62672cb1,"trưa ghé đây ăn , , tô bò gân , gân thì không ...",0


# Extract texts

We extract the text-only part of our dataset and examine them:

In [0]:
all_texts = list(train_df['text'])

for text in all_texts[:10]:
  print(text + '\n')

Đến quán 2 lần thôi , rất là thích !
Quán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .
Nhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .
Thực_đơn phong_phú đa_dạng , giá siêu rẻ .
Thức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .
Desert ngọt vừa , không quá ngấy , rất ngon .
Trà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .
Nói_chung điểm 10 cho chất_lượng , thích quán lắm !

Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz

Phục_vụ lâu quá mặc_dù khách rất vắng .
Đợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .

Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .

Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .
Bánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . 

This in comparison with data for our translation task (IWSLT'15) is in fact drastically different! 

To see that, let's examine the test data from IWSLT'15:

In [0]:
!head -n 32 $mount_point/raw/tst2013.vi

Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;
Tôi đã rất tự hào về đất nước tôi .
Ở trường , chúng tôi dành rất nhiều thời gian để học về cuộc đời của chủ tịch Kim II- Sung , nhưng lại không học nhiều về thế giới bên ngoài , ngoại trừ việc Hoa Kỳ , Hàn Quốc và Nhật Bản là kẻ thù của chúng tôi .
Mặc dù tôi đã từng tự hỏi không biết thế giới bên ngoài kia như thế nào , nhưng tôi vẫn nghĩ rằng mình sẽ sống cả cuộc đời ở BắcTriều Tiên , cho tới khi tất cả mọi thứ đột nhiên thay đổi .
Khi tôi lên 7 , tôi chứng kiến cảnh người ta xử bắn công khai lần đầu tiên trong đời , nhưng tôi vẫn nghĩ cuộc sống của mình ở đây là hoàn toàn bình thường .
Gia đình của tôi không nghèo , và bản thân tôi thì chưa từng phải chịu đói .
Nhưng vào một ngày của năm 1995 , mẹ tôi mang về nhà một lá thư từ một người chị em cùng chỗ làm với mẹ .
Trong đó có viết : Khi chị đọc được những dòng này thì cả gia đình 

What you are seeing here is called "domain mismatch", where the distribution of the two datasets are (wildly) different. In the reviews, you might see:



```
* Idioms "Cá lớn nuốt cá bé" "Hãy buông đôi tay nhau ra" "Không yêu đừng nói lời cay đắng"
* New words "Gấu mình dẫn đi ăn. "
* Short-hand writing "đồ ăn ko dc ngon lm đâu"
* Teen code "HjHj ch0 n4y 4n k dc ngon l4'm :3:3"
* Capitalization "TÔI KHÔNG THÍCH ĂN QUÁN NÀY cHúT nàO!!"
* Tone marks "toi ko thich an quan nay chut nao!!"
* Grammatical error.
```

While you can hardly find such instances in the translation training data IWSLT'15. Expectedly the translation models won't be able to understand such instances and give low quality translation.

To somewhat bridge this gap in domain mismatch, let's perform some preprocessing to our Foody Sentiment Analysis dataset. This should include special character filtering, lower-casing, etc.

In [0]:
# Removes parentheses etc., and leaves only alphanumeric characters
strip_special_chars = re.compile("[^\w0-9 .,:!?]+")

def clean_sentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())


def back_translate_preprocess(text):
  # We only use the first 32 words on each line:
  lines = []
  for line in text.split('\n'):
    line = ' '.join(line.split()[:32])
    line = clean_sentences(line)
    lines.append(line)
  text = '\n'.join(lines)
  
  # Replace the _ character with a proper space.
  text = text.replace('_', ' ')
  text = text.strip()

  # Since there is a lot of upper-casing in reviews,
  # e.g. QUÁN NÀY LÀM ĂN QUÁ TỆ, ĐỒ ĂN KHÔNG SẠCH SẼ!!!
  # We might need to lower-case all the text.
  text = text.replace('Đ', 'đ')

  text = text.replace('E', 'e')
  text = text.replace('È', 'è')
  text = text.replace('É', 'é')
  text = text.replace('Ẻ', 'ẻ')
  text = text.replace('Ẽ', 'ẽ')
  text = text.replace('Ẹ', 'ẹ')

  text = text.replace('Ê', 'ê')
  text = text.replace('Ề', 'ề')
  text = text.replace('Ế', 'ế')
  text = text.replace('Ể', 'ể')
  text = text.replace('Ễ', 'ễ')
  text = text.replace('Ệ', 'ệ')

  text = text.replace('A', 'a')
  text = text.replace('À', 'à')
  text = text.replace('Á', 'á')
  text = text.replace('Ả', 'ả')
  text = text.replace('Ã', 'ã')
  text = text.replace('Ạ', 'ạ')

  text = text.replace('Ă', 'ă')
  text = text.replace('Ằ', 'ằ')
  text = text.replace('Ắ', 'ắ')
  text = text.replace('Ẳ', 'ẳ')
  text = text.replace('Ẵ', 'ẵ')
  text = text.replace('Ặ', 'ặ')

  text = text.replace('Â', 'â')
  text = text.replace('Ầ', 'ầ')
  text = text.replace('Ấ', 'ấ')
  text = text.replace('Ẫ', 'ẫ')
  text = text.replace('Ẩ', 'ẩ')
  text = text.replace('Ậ', 'ậ')

  text = text.replace('U', 'u')
  text = text.replace('Ù', 'ù')
  text = text.replace('Ú', 'ú')
  text = text.replace('Ủ', 'ủ')
  text = text.replace('Ũ', 'ũ')
  text = text.replace('Ụ', 'ụ')

  text = text.replace('Ư', 'ư')
  text = text.replace('Ừ', 'ừ')
  text = text.replace('Ứ', 'ứ')
  text = text.replace('Ử', 'ử')
  text = text.replace('Ữ', 'ữ')
  text = text.replace('Ự', 'ự')

  text = text.replace('O', 'o')
  text = text.replace('Ò', 'ò')
  text = text.replace('Ó', 'ó')
  text = text.replace('Ỏ', 'ỏ')
  text = text.replace('Õ', 'õ')
  text = text.replace('Ọ', 'ọ')
  
  text = text.replace('Ơ', 'ơ')
  text = text.replace('Ờ', 'ờ')
  text = text.replace('Ớ', 'ớ')
  text = text.replace('Ở', 'ở')
  text = text.replace('Ỡ', 'ỡ')
  text = text.replace('Ợ', 'ợ')
  
  text = text.replace('Ô', 'ô')
  text = text.replace('Ồ', 'ồ')
  text = text.replace('Ố', 'ố')
  text = text.replace('Ổ', 'ổ')
  text = text.replace('Ỗ', 'ỗ')
  text = text.replace('Ộ', 'ộ')

  # Some other simple standardization.
  text = text.replace('ko ', 'không ')
  text = text.replace('dc ', 'được ')
  text = text.lower()
  return text + '\n'
  

# Write the result to a text file:
train_text_file = os.path.join(foody_data_dir, 'foody_processed_train.txt')
review_lens = []  # number of lines in each review.
with open(train_text_file, 'w') as f:
  for text in all_texts:
    processed_text = back_translate_preprocess(text)
    f.write(processed_text + '\n')
    num_lines = processed_text.count('\n')
    review_lens.append(num_lines)
    
    
!head -n 20 $train_text_file

OSError: ignored

In [0]:
!head -n 20 $train_text_file

đến quán 2 lần thôi , rất là thích !
quán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng rãi thoáng mát sạch sẽ .
nhân viên phục vụ nhiệt tình , siêu dễ thương .
thực đơn phong phú đa dạng , giá siêu rẻ .
thức ăn nêm nếm hợp khẩu vị , ngon miệng .
desert ngọt vừa , không quá ngấy , rất ngon .
trà sữa , uyên ương thơm và đúng vị mình đã từng uống ở hongkong .
nói chung điểm 10 cho chất lượng , thích quán lắm !

đến quán vào tối chủ nhật . có band hát . khá ổn . tuy nhiên không gian tầng 1 hơi hẹp . ngồi cả ra vỉa hè . đồ uống không ngon lămz

phục vụ lâu quá mặc dù khách rất vắng .
đợi gần 15p mới mang được ly cà phê ra . ông này chỉ khéo quảng bá chứ với mình chắc không quay lại nữa .

không gian bé tí , quán chật chội , đông người nên phục vụ chưa tốt . đồ uống bình thường  đặc biệt khi so với giá cả  và không có gì đặc sắc cả .

khi mình order , đặt bánh thì nhận được sự tiếp đãi , tư vấn và nói chuyện rất gần gũi , rất nhiệt tình .
bánh thì rất tuyệt , nhân béo , rau câu không dùng 

# Pull or Clone code from our VietAI's Github `vietai/back_translate`

In [0]:
src = '/content/back_translate'
if not os.path.exists(src):
  !git clone https://github.com/vietai/back_translate.git
else:
  % cd /content/back_translate
  !git pull
  % cd /
  
!ls $src

Cloning into 'back_translate'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 76 (delta 43), reused 53 (delta 23), pack-reused 0
Unpacking objects: 100% (76/76), done.
back_translate.py  LICENSE	README.md	t2t_decoder.py
decoding.py	   problems.py	t2t_datagen.py	t2t_trainer.py


# Back-translate

You can surely perform beam search decoding one sentence after another until finish. This, however, would take a very long time. Our code helps you with handling all possible speedups you can make:

* Decoding on ML accelerator hardware.
* Batch processing of sentences.
* Processing sentences of similar length at the same time.
* Saving intermediate results during the process.

At its core, the `back_translate.py` script essentially packages two calls to `t2t-decode` from vi -> en and then en -> vi. In the code, we added the ability to save intermediate results after every single batch of data since back-translating the full dataset can take quite a long time.

In [0]:
output_file = os.path.join(foody_data_dir, 'foody_paraphrased_train.txt')

is_demo = True  #@param {type:"boolean"}
if is_demo:
  paraphrase_from_file = os.path.join(foody_data_dir, 'demo_train.txt')
  paraphrase_to_file = os.path.join(foody_data_dir, 'demo_out.txt')
  # Remove previous results to run again.
  tmp_file = paraphrase_from_file + '.tmp.en.txt'
  if tf.gfile.Exists(tmp_file):
    tf.gfile.Remove(tmp_file)
  if tf.gfile.Exists(paraphrase_to_file):
    tf.gfile.Remove(paraphrase_to_file)
else:
  paraphrase_from_file = train_text_file
  paraphrase_to_file = output_file


# Without specifying batch_size in decode_hparams, the default value will be 32.
!python $src/back_translate.py \
--lang=vi \
--model=transformer --hparams_set=transformer_tiny \
--decode_hparams="beam_size=4,alpha=0.6" \
--paraphrase_from_file=$paraphrase_from_file \
--paraphrase_to_file=$paraphrase_to_file \
--output_dir=/dummy/ \
--use_tpu=$use_tpu \
--cloud_tpu_name=$tpu_address

W0803 09:04:47.818745 140674679416704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0803 09:04:50.068230 140674679416704 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0803 09:04:51.642825 140674679416704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 09:04:51.643320 140674679416704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/d

# Print some paraphrases

In [0]:
!head -n 10 $train_text_file
print('==' * 10)
!head -n 10 $output_file

đến quán 2 lần thôi , rất là thích !
quán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng rãi thoáng mát sạch sẽ .
nhân viên phục vụ nhiệt tình , siêu dễ thương .
thực đơn phong phú đa dạng , giá siêu rẻ .
thức ăn nêm nếm hợp khẩu vị , ngon miệng .
desert ngọt vừa , không quá ngấy , rất ngon .
trà sữa , uyên ương thơm và đúng vị mình đã từng uống ở hongkong .
nói chung điểm 10 cho chất lượng , thích quán lắm !

đến quán vào tối chủ nhật . có band hát . khá ổn . tuy nhiên không gian tầng 1 hơi hẹp . ngồi cả ra vỉa hè . đồ uống không ngon lămz
Cùng lúc đó , nó giống như vậy !
Đó là nó .
Những quốc gia siêu dễ thương .
Nó rất đa dạng , rất rẻ .
Nó 񡩿nhăn mật khẩu , ngon .
sa mạc chỉ ngọt , không tốt lắm .
Loại trà sữa , ward và nó thật sự là bạn có 〓
Đó là khoảng 10 đến chất lượng , như vậy !

Đây là một đêm Sunngày . Có 39s một ban nhạc tuyệt vời . Nó khá ổn định . Nhưng không gian của một chút , ngồi trên vỉa hè . Nó không gian dài . Nó không ngon .


# Convert back-translated text to CSV format.

First, we have to process `output_file` back into a list of reviews. Since each review can have multiple lines of text, we make use of `review_lens` recorded previously to group the lines in `output_file` back into individual reviews.

In [0]:
# First we read the text in output_file
with open(output_file, 'r') as f:
  output_lines = f.readlines()

# Now we group lines into individual reviews according to review_lens.
bt_reviews = []  # bt == 'back translated'
for review_len in review_lens:
  review = output_lines[:review_len]
  output_lines = output_lines[review_len+1:]
  # combine the lines.
  review = ''.join(review)
  bt_reviews.append(review)

Next, we put the list of reviews `bt_reviews` back into Pandas data frame and save it back to csv format.

In [0]:
bt_train_df = train_df.copy()   # bt == back translated
bt_train_df['text'] = bt_reviews

bt_train_csv = os.path.join(foody_data_dir, 'bt_train.csv')
bt_train_df.to_csv(bt_train_csv, index=False)

Done! Let's load the back-translated dataset and check to see if everything looks okay:

In [0]:
bt_train_df = pd.read_csv(bt_train_csv)
bt_train_df

,id,text,class
0,14feb4e5,"Cùng lúc đó , nó giống như vậy !\nĐó là nó .\n...",1
1,9c6f6036,Đây là một đêm Sunngày . Có 39s một ban nhạc t...,0
2,0f462cfe,"Nó dài 3999s , mặc dù cuộc sống rất khó chịu ....",0
3,741fd21b,"Nó có một chút không gian rất nhỏ , chứa đựng ...",0
4,5cd6dbbc,"Khi bạn \3999999m , vấn đề , vấn đề và nói rất...",1
5,a9b0679b,"Có một thời gian an toàn ở đây . Gần đây , 399...",0
6,d0919c4a,Nó đẹp . Nó rất tốt . Nó rất tốt . Nó rất già ...,1
7,a7dd3e51,"Gemini , âm nhạc : 3 cái lọ đang trên mái nhà ...",1
8,4dc3df78,"Và bạn vào buổi sáng của bạn , 83 . Bạn thấy a...",0
9,62672cb1,"chiều hôm nay , con bò này rất khó , nó không ...",0


# Test Time Augmentation (TTA).

If we have access to the back-translation models (vi->en & en->vi), can we make use of them during test time?

The answer is yes, one idea called "Test Time Augmentation" asks the model to average its predictions on real and synthetic examples, generated from the real ones. This technique is employed widely in competitions and reliably gives improvement on Computer Vision benchmarks.

To use TTA, here we simply repeat the above steps on the test data. Note that the release test data here does not have the `class` column so we are not cheating for using TTA. We have generated the back-translated test data in a separate colab using the same procedure presented so far. The result can be found in the same Cloud Storage:

In [0]:
bt_test_csv = os.path.join(foody_data_dir, 'bt_test.csv')
bt_test_df = pd.read_csv(bt_test_csv)
bt_test_df

,id,text
0,3afca6c4,"Đó là khoảng 6 loại khác nhau , mỗi thứ đều ăn..."
1,a52dd7db,Đó là từ nhà của nhà . Nó sẽ không thay đổi từ...
2,a95ec3a0,Bạn đi ăn đúng và đi !\nNó từ trước khi chỉ là...
3,68c1c84a,"Nó gần như như là ruột của bạn , nó luôn luôn ..."
4,627bbd02,"Đó là khoảng 25 năm cuộc sống ở phương Tây , đ..."
5,2cbdb995,Nó luôn là cách .\nWilk &quot; Milk &quot; Mil...
6,5b92db71,"Nhà hàng đẹp , không gian rộng , có một căn ph..."
7,80324579,"Và sau đó có thể ăn , tôi chọn một cái sân nhà..."
8,0a237bd7,Nó rất ngon .\nĐiều yêu thích nhất là cái fato...
9,e660b38c,Đó là về những con cá mập khổng lồ được lấp đầ...


# Now let's train a model on the augmented data.

Using Machine Learning to solve this dataset is a standard assignment in our VietAI's foundation class. We use methods/solution/code from this assignment, prepared by VietAI's teaching staff and is available to any student of the class. 

At the moment this material is not readily viewable to the public, but we are working towards that! For now we report the results obtained from training the solution model on the original dataset and the augmented dataset (column "with BT") here. We did not make any extra modification from the original solution and still got improvement right away! Here we report the results for two settings: (1) limited data where only 10K training examples is available, and (2) when there is a full access to 27K training examples.


|     | Normal  | with BT| with TTA |
|-----|---------|-----------------------|------------------------|
| 27k | 85.86% | 86.05%               | __86.95%__                |
| 10k | 83.48% | __85.91%__               | 85.05%                |

# Acknowledgements

This work is made possible by [VietAI](http://vietai.org/). Special thanks to [Thang Luong](http://thangluong.com), Le Cao Thang, and Hoang Quy Phat for collaborating and giving comments.

# References

1. Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems. 2017.

2. Vaswani, Ashish, et al. "Tensor2tensor for neural machine translation." arXiv preprint arXiv.

3. Sennrich, Rico, Barry Haddow, and Alexandra Birch. "Improving neural machine translation models with monolingual data." arXiv preprint arXiv:1511.06709 (2015).

4. Edunov, Sergey, et al. "Understanding back-translation at scale." arXiv preprint arXiv:1808.09381 (2018).